# Verification forces interfaciales en 2D et 3D

In [ ]:
from trustutils import run
run.introduction("Corentin Reiss")

In [ ]:
run.reset()
run.initBuildDirectory()

sondes = {"2D":{}, "3D":{}}

n_seg_sonde = 40
X = 0.1
Y = 1

tab_name_sondes = ["grdp", "diff", "conv", "drag", "lift", "disp", "lub", "vorticite", "gradient_vitesse_liquide_eau" ]

x0 = X/(2*n_seg_sonde)
y0 = Y/(2*n_seg_sonde)

x1 = X*(2*n_seg_sonde-1)/(2*n_seg_sonde)
y1 = Y*(2*n_seg_sonde-1)/(2*n_seg_sonde)

str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde} {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x0} {y0}  {x1} {y1}  \n"

sondes["2D"] = str_sonde

str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde} {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x0} {y0} .5 {x1} {y1} .5 \n"

sondes["3D"] = str_sonde

# Refinement
name_mesh  = { "2D" : "Mailler dom \n { \n Pave block  \n { \n Origine 0. 0.    \n Nombre_de_Noeuds 41 41   \n  Longueurs  0.1 1.    \n } \n { \n bord symetrie    X = 0   0 <= Y <= 1.  \n bord top   Y = 1.   0 <= X <= 0.1  \n bord bottom    Y = 0   0 <= X <= 0.1 \n bord wall   X = 0.1 0 <= Y <= 1. \n } \n } " ,
               "3D" : "Mailler dom \n { \n Pave block  \n { \n Origine 0. 0. 0. \n Nombre_de_Noeuds 41 41 2 \n  Longueurs  0.1 1. 1. \n } \n { \n bord symetrie    X = 0   0 <= Y <= 1. 0 <= Z <= 1.  \n bord bottom   Y = 0. 0 <= X <= 0.1   0 <= Z <= 1.   \n bord top  Y = 1. 0 <= X <= 0.1   0 <= Z <= 1.   \n bord symetrie   Z = 1.   0 <= X <= 0.1  0 <= Y <= 1. \n bord symetrie    Z = 0   0 <= X <= 0.1  0 <= Y <= 1. \n bord wall   X = 0.1  0 <= Y <= 1 0 <= Z <= 1.  \n } \n }  "  }   

polyedriser = { "2D" : { "VDF" : " ",  "PolyMAC" : " "},  "3D" : { "VDF" : " ",  "PolyMAC" : "polyedriser dom"} }

schema = { "VDF" : " vdf  dis option_vdf { all_options } ", 
           "PolyMAC" : " PolyMAC_P0  dis option_PolyMAC { traitement_axi } "  }

dimension = { "2D" : 2, "3D" : 3 }

u_0 = { "2D" : " 4 0 0 1-10*x*x 2-20*x*x ", "3D":" 6 0 0 1-10*x*x 2-20*x*x 0 0 " }

for d in dimension.keys():
    for s in schema.keys() :
        substitutions_dict = {"dimension" : dimension[d],
                                "schema" : schema[s],
                                "name_mesh" : f'{name_mesh[d]}' ,
                                "polyedriser" : polyedriser[d][s],
                                "u_0"  : u_0[d] ,
                                "sondes" : sondes[d]
                            }

        run.addCaseFromTemplate("jdd.data",targetDirectory=f"{d}_{s}",dic=substitutions_dict)

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from trustutils import plot

# Calculer les résultats analytiques
db = 0.04
beta_vm = .5
cd = .3 
cl = 0.2
d_td = 0.1

xa = np.linspace( 0,.1,1000 )
ya = np.linspace( 0,1.,1000 )

alpa = .5*ya
ulya = 1-10*xa*xa 
ugya = 2-20*xa*xa
rhol = 1.e3
rhog = 1.e2
mul = 1.e-3
dx_ulya = -10*2*xa
dx2_ulya = -10*2
wlz = dx_ulya
dy_alpa = .5
Cw1_ = -.1 
Cw2_ = .147
pa  = 1.e5
gpa = 0

drag_gy_a = - 3. / 4. * cd * alpa / db * rhol * (ugya-ulya)**2 / (alpa*rhog)
lift_gx_a = - alpa * rhol * cl * (ugya-ulya) * wlz / (alpa*rhog)
disp_gy_a = - rhol * d_td * (ugya-ulya)**2 * dy_alpa / (alpa*rhog)
lub_gx_a = - 2. * alpa * rhol * (ugya-ulya)**2 / db * np.maximum(alpa-alpa, Cw1_ + Cw2_*db/(2.*(.1-xa))) / (alpa*rhog)
gp_gy_a = - alpa * gpa / (alpa*rhog)
conv_gy_a = alpa - alpa
diff_ly_a = dx2_ulya * mul * (1-alpa) / ((1-alpa)*rhol)

plt_symbol = { "2D" : { "VDF" : "xb",  "PolyMAC" : "1r" },  "3D" : { "VDF" : "+m",  "PolyMAC" : "3c" } }

fig = plt.figure(figsize = (16,8))
axs = fig.subplots(2,4)

xs = np.linspace(X/(2*n_seg_sonde), X*(2*n_seg_sonde-1)/(2*n_seg_sonde), n_seg_sonde)

for d in dimension.keys():
    for s in schema.keys() :
        name = f"{d}_{s}"

        drag_gy_s=np.array(plot.loadText(f"{name}/jdd_DRAG.son"))[4::2*dimension[d],-1]
        lift_gx_s=np.array(plot.loadText(f"{name}/jdd_LIFT.son"))[2::2*dimension[d],-1]
        disp_gy_s=np.array(plot.loadText(f"{name}/jdd_DISP.son"))[4::2*dimension[d],-1]
        lub_gx_s =np.array(plot.loadText(f"{name}/jdd_LUB.son"))[2::2*dimension[d],-1]
        gp_gy_s  =np.array(plot.loadText(f"{name}/jdd_GRDP.son"))[4::2*dimension[d],-1]
        conv_gy_s=np.array(plot.loadText(f"{name}/jdd_CONV.son"))[4::2*dimension[d],-1]
        diff_ly_s=np.array(plot.loadText(f"{name}/jdd_DIFF.son"))[3::2*dimension[d],-1]

        axs[0,0].plot([-1,-1],[-1,-1], plt_symbol[d][s], label = name)
        axs[0,1].plot(xs, gp_gy_s, plt_symbol[d][s])
        axs[0,2].plot(xs, conv_gy_s, plt_symbol[d][s])
        axs[0,3].plot(xs, diff_ly_s, plt_symbol[d][s])
        axs[1,0].plot(xs, drag_gy_s, plt_symbol[d][s])
        axs[1,1].plot(xs, lift_gx_s, plt_symbol[d][s])
        axs[1,2].plot(xs, disp_gy_s, plt_symbol[d][s])
        axs[1,3].plot(xs, lub_gx_s, plt_symbol[d][s])

axs[0,0].plot([-1, -1], [-1, -1], "k-", label = "analytic")
axs[0,1].plot(xa, gp_gy_a, "k-")
axs[0,2].plot(xa, conv_gy_a, "k-")
axs[0,3].plot(xa, diff_ly_a, "k-")
axs[1,0].plot(xa, drag_gy_a, "k-")
axs[1,1].plot(xa, lift_gx_a, "k-")
axs[1,2].plot(xa, disp_gy_a, "k-")
axs[1,3].plot(xa, lub_gx_a, "k-")

axs[0,0].legend(loc = "center")
axs[0,0].set_axis_off()
axs[0,0].set_xlim([0,1])
axs[0,1].set_title("Pressure gradient")
axs[0,2].set_title("Convection")
axs[0,3].set_title("Diffusion")
axs[0,3].set_ylim(-2.5e-5,1.e-5)
axs[1,0].set_title("Drag")
axs[1,1].set_title("Lift")
axs[1,1].set_ylim(0,5)
axs[1,2].set_title("Turbulent Dispersion")
axs[1,2].set_ylim(-40,0)
axs[1,3].set_title("Wall Lubrication")
axs[1,3].set_ylim(-400,0)

fig.tight_layout()